<a href="https://colab.research.google.com/github/iamshirdi/Datascience/blob/master/python/sqlite_itunes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h4>parse an iTunes Music Library XML list of albums, artists, and Genres and produce a properly normalized database using a Python program.

In [1]:
!rm -r *
!wget http://www.py4e.com/code3/tracks.zip && unzip tracks.zip
!mv ./tracks/* ./ && ls

--2019-06-18 17:49:51--  http://www.py4e.com/code3/tracks.zip
Resolving www.py4e.com (www.py4e.com)... 104.27.158.166, 104.27.159.166, 2606:4700:30::681b:9fa6, ...
Connecting to www.py4e.com (www.py4e.com)|104.27.158.166|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.py4e.com/code3/tracks.zip [following]
--2019-06-18 17:49:51--  https://www.py4e.com/code3/tracks.zip
Connecting to www.py4e.com (www.py4e.com)|104.27.158.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52332 (51K) [application/zip]
Saving to: ‘tracks.zip’

tracks.zip          100%[===================>]  51.11K   287KB/s    in 0.2s    

2019-06-18 17:49:52 (287 KB/s) - ‘tracks.zip’ saved [52332/52332]

Archive:  tracks.zip
   creating: tracks/
  inflating: tracks/tracks.py        
  inflating: tracks/README.txt       
  inflating: tracks/Library.xml      
Library.xml  README.txt  tracks  tracks.py  tracks.zip


In [2]:
!cat Library.xml

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
	<key>Major Version</key><integer>1</integer>
	<key>Minor Version</key><integer>1</integer>
	<key>Date</key><date>2015-11-24T11:12:10Z</date>
	<key>Application Version</key><string>12.3.1.23</string>
	<key>Features</key><integer>5</integer>
	<key>Show Content Ratings</key><true/>
	<key>Music Folder</key><string>file:///Users/csev/Music/iTunes/iTunes%20Music/</string>
	<key>Library Persistent ID</key><string>B7006C9E9799282E</string>
	<key>Tracks</key>
	<dict>
		<key>369</key>
		<dict>
			<key>Track ID</key><integer>369</integer>
			<key>Name</key><string>Another One Bites The Dust</string>
			<key>Artist</key><string>Queen</string>
			<key>Composer</key><string>John Deacon</string>
			<key>Album</key><string>Greatest Hits</string>
			<key>Genre</key><string>Rock</string>
			<key>Kind</key><string>MPEG audio f

In [3]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('itunesdb.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
DROP TABLE IF EXISTS Genre;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')

# inserting xml data

fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'Library.xml'

  
# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key : #key is name or artist etc
            found = True
    return None
  

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict') #each dict has key and type returned as list
print('Dict count:', len(all))

for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name') #lookup above function finds the xml tag text
    artist = lookup(entry, 'Artist')
    genre=lookup(entry, 'Genre')
    album = lookup(entry, 'Album')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')

    if name is None or artist is None or album is None or genre is None : 
        continue

    print(name, artist, album,genre, count, rating, length)

    cur.execute('''INSERT OR IGNORE INTO Genre (name) 
        VALUES ( ? )''', ( genre, ) ) #autoincrment assigns id automatically so no need to specify 
    cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cur.fetchone()[0]

    
    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) ) #autoincrment assigns id automatically so no need to specify 
    #ignore is ignored because of unique if already exists id and artist

    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]


    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]
#replace is update

    cur.execute('''INSERT OR REPLACE INTO Track 
        (title, album_id,genre_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ?,? )''', 
        ( name, album_id,genre_id, length, rating, count ) )
    conn.commit()



Enter file name: 
Dict count: 404
Another One Bites The Dust Queen Greatest Hits Rock 55 100 217103
Asche Zu Asche Rammstein Herzeleid Industrial 79 100 231810
Beauty School Dropout Various Grease Soundtrack 48 100 239960
Black Dog Led Zeppelin IV Rock 109 100 296620
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] Rock 33 100 87118
Circles Bryan Lee Blues Is Funk 54 60 355369
Comfortably Numb Pink Floyd The Wall [Disc 2] Rock 36 100 384130
Crazy Little Thing Called Love Queen Greatest Hits Rock 38 100 163631
Electric Funeral Black Sabbath Paranoid Metal 44 100 293015
Fat Bottomed Girls Queen Greatest Hits Rock 38 100 257515
For Those About To Rock (We Salute You) AC/DC Who Made Who Rock 84 100 353750
Four Sticks Led Zeppelin IV Rock 84 100 284421
Furious Angels Rob Dougan The Matrix Reloaded Soundtrack 54 100 330004
Gelle Bryan Lee Blues Is Blues/R&B 45 60 199836
Going To California Led Zeppelin IV Rock 100 100 215666
Grease Various Grease Soundtrack 42 100 205792
Hand of Doom Bl

In [4]:
sqlstr='''SELECT Track.title, Artist.name, Album.title, Genre.name 
    FROM Track JOIN Genre JOIN Album JOIN Artist 
    ON Track.genre_id = Genre.ID and Track.album_id = Album.id 
        AND Album.artist_id = Artist.id
    ORDER BY Artist.name LIMIT 3'''


for row in cur.execute(sqlstr):
    print(str(row[0]), row[1], row[2], row[3])


For Those About To Rock (We Salute You) AC/DC Who Made Who Rock
Hells Bells AC/DC Who Made Who Rock
Shake Your Foundations AC/DC Who Made Who Rock


In [5]:
! curl --upload-file ./itunesdb.sqlite https://transfer.sh/itunesdb.sqlite

https://transfer.sh/HTZdE/itunesdb.sqlite